In [ ]:
import sqdtoolz as stz
import numpy as np

lab = stz.Laboratory(instr_config_file = "example_tabor.yaml", save_dir = "Data/", using_VS_Code=True)

lab.cold_reload_last_configuration()
#lab.open_browser()

This is a simple notebook to characterise qubits using the Tabor PXI unit.

Setup assumes:
- Daisy-chained SGS100As with Rb oven connected to all relevant instruments
- DDG `AB` connected to Tabor's `TRIG 1`
- Tabor's `CH1 M1` connected to src_res's `TRIG`
- Using QBox with QuinStar amplifier on its labelled ports
- Using a splitter to combine charge line output from QBox and the readout output (i.e. both labelled as 'to fridge') to feed into the input feedline of the device
- Tabor's `CH1` and `CH2` outputs connect to QBox along with the Stahl for DC
- ACQ output from QBox goes into `CH1` of the Tabor on the input side
- The Tabor is internally triggered, so it does not need anything connected to `TRIG IN`

In [ ]:
lab.load_instrument('stahl_volt')
volt_module1 = stz.GENvoltSource('flux_coil', lab, ['stahl_volt', 'CH3'])
volt_module2 = stz.GENvoltSource('vFlux', lab, ['stahl_volt', 'CH4'])
volt_module3 = stz.GENvoltSource('DCoff_I', lab, ['stahl_volt', 'CH1']) #stahl_volt was loaded earlier
volt_module4 = stz.GENvoltSource('DCoff_Q', lab, ['stahl_volt', 'CH2'])

#load the tabor unit for data acquisition
lab.load_instrument('Tabor')
stz.ACQ('TaborACQ', lab, ['Tabor', 'ACQ'])

#load the resonator microwave source
lab.load_instrument('src_res')
stz.GENmwSource('mw_res', lab, 'src_res', 'RFOUT')

#load the qubit microwave source
lab.load_instrument('src_qubit')
stz.GENmwSource('mw_qubit', lab, 'src_qubit', 'RFOUT')

#load the down-conversion microwave source
lab.load_instrument('src_dnc')
stz.GENmwSource('mw_dnc', lab, 'src_dnc', 'RFOUT')

#load the Digital Delay Generator (DDG)
lab.load_instrument('DDG')
stz.DDG('DDG', lab, 'DDG')

lab.load_instrument('sw_rpi')
stz.GENswitch('swQBox', lab, 'sw_rpi')

**Keep an eye on this and turn on the fans if required!**

In [77]:
#Temperature Check
lab.HAL('TaborACQ')._instr_acq._parent._get_cmd(':SYST:MEAS:TEMP?')

'60.551800, 3'

# Resonator Spectroscopy

In [ ]:
#set experiment repition time
lab.HAL("DDG").RepetitionTime = 5e-6  

# some settings for the DDG to trigger Tabor
lab.HAL('DDG').get_trigger_output('AB').TrigPulseLength = 50e-9
lab.HAL('DDG').get_trigger_output('AB').TrigPolarity = 1
lab.HAL('DDG').get_trigger_output('AB').TrigPulseDelay = 0
lab.HAL('DDG').get_trigger_output('AB').TrigEnable = True 

#Set up ACQ for Tabor
num_of_repetitions = 8192
num_of_segments = 1
num_of_samples = 5040
lab.HAL('TaborACQ').set_acq_params(num_of_repetitions,num_of_segments,num_of_samples) #num of repititions, num of segments, num of samples
lab.HAL('TaborACQ').SampleRate = 2.5e9 #2.5 Gigasamples per second
lab.HAL('TaborACQ').ChannelStates = (True, True) #(first channel, second channel)

#set up AWG in Tabor
#AWG sample rate must be 80% of ACQ sample rate
AWG_sample_rate = 2.0e9

stz.WaveformAWG("wfm_res", lab,  [(['Tabor', 'AWG'], 'CH1'), (['Tabor', 'AWG'], 'CH2')], AWG_sample_rate, total_time = 4e-6)
lab.HAL("wfm_res").set_valid_total_time(4e-6)
lab.HAL("wfm_res").clear_segments()
lab.HAL("wfm_res").add_waveform_segment(stz.WFS_Constant("Wait", None, 512e-9, 0.0)) #possibly kill
lab.HAL("wfm_res").add_waveform_segment(stz.WFS_Constant("Read", None, -1, 0.0))
lab.HAL("wfm_res").get_output_channel(0).marker(2).set_markers_to_segments(['Read'])
lab.HAL("wfm_res").set_trigger_source_all(lab.HAL("DDG").get_trigger_output('AB'))
lab.HAL("TaborACQ").set_trigger_source(lab.HAL("wfm_res").get_output_channel(0).marker(2))

#set up digital signal processing using the Tabor's FPGA
stz.ProcessorFPGA('fpga_dsp', lab)
lab.PROC('fpga_dsp').reset_pipeline()
lab.PROC('fpga_dsp').add_stage(stz.FPGA_DDCFIR([[{'fLO':25e6, 'fc':1e6, 'Taps':128}]]))
lab.PROC('fpga_dsp').add_stage(stz.FPGA_Decimation('sample', 10))
lab.PROC('fpga_dsp').add_stage(stz.FPGA_Integrate('sample'))
lab.PROC('fpga_dsp').add_stage(stz.FPGA_Integrate('repetition'))
lab.HAL("TaborACQ").set_data_processor(lab.PROC('fpga_dsp'))

#measurement time
meas_time = (num_of_samples / lab.HAL("TaborACQ").SampleRate)
total_exp_time = meas_time * num_of_repetitions 

#Set-up Local Oscillatior source for Down Conversion
lab.HAL('mw_dnc').Mode = 'Continuous'
lab.HAL('mw_dnc').Power = 16
lab.HAL('mw_dnc').Frequency = 8.1e9
lab.HAL('mw_dnc').Output = True
lab.HAL('mw_dnc').ManualActivation = True

#Set-up Resonator microwave source 
lab.HAL('mw_res').Mode = 'Continuous'
lab.HAL('mw_res').Power = -30
lab.HAL('mw_res').Frequency = 8.1e9
lab.HAL('mw_res').Output = True

#QBox set to measurement, PMeas = measure, Pmix = mixer calibration 
#lab.HAL('swQBox').Position = "Pmeas"

#Create the experimental configuration object
stz.ExperimentConfiguration('contMeas', lab, lab.HAL("DDG").RepetitionTime, ['DDG', 'mw_res', 'mw_dnc', 'wfm_res'], 'TaborACQ')
lab.CONFIG('contMeas').plot();
lab.update_state()

In [ ]:
#Variable for MW res frequency
stz.VariableProperty('res_freq_initial', lab, lab.HAL("mw_res"), 'Frequency')

#Variable for Local Oscillator frequency
stz.VariableProperty('dnc_freq', lab, lab.HAL("mw_dnc"), 'Frequency')

#Assign a fixed variable spacing between MWFreq + 25e6 = MWDownConv
stz.VariableSpaced('res_freq', lab, 'res_freq_initial', 'dnc_freq', 25e6)

#ASsign flux variable
stz.VariableProperty('vFlux', lab, lab.HAL('vFlux'), 'Voltage') #flux line
stz.VariableProperty('flux_coil', lab, lab.HAL('flux_coil'), 'Voltage') #flux coil

#Variable for power supplied to the microwave resonator
stz.VariableProperty('res_power', lab, lab.HAL("mw_res"), 'Power')

stz.ExperimentSpecification('resonator', lab, 'Cavity')


In [ ]:
exp = stz.Experiment(name = f'res_Q1', expt_config = lab.CONFIG('contMeas'))
leData = lab.run_single(exp, [(lab.VAR('res_freq'), np.arange(7.4e9,7.42e9, 0.25e6))]);

In [ ]:
exp = stz.Experiment(name = f'flux_Q1', expt_config = lab.CONFIG('contMeas'))
leData = lab.run_single(exp, [(lab.VAR('vFlux'), np.arange(-1.3,-0.6,0.03)), (lab.VAR('res_freq'), np.arange(7.4e9,7.42e9, 0.5e6))]);

In [51]:
lab.VAR('vFlux').Value = -0.966

In [ ]:
# stz.ExperimentSpecification('res_Q1', lab, 'Cavity')
# lab.SPEC('res_Q1').set_destination('Frequency', lab.VAR('res_freq'))
# lab.SPEC('res_Q1').set_destination('Power', lab.HAL('mw_res'), 'Power')

In [ ]:
from sqdtoolz.Experiments.Experimental.ExpPeakScouterIQ import ExpPeakScouterIQ

#current resonator to analyse

#create experiment for resonator spectroscopy
res_spec = ExpPeakScouterIQ('res_spec', lab.CONFIG('contMeas'), 
                    param_centre=lab.SPEC('res_Q1')['Frequency'], param_width=lab.SPEC('res_Q1')['LineWidth'], 
                    is_trough=True, fit_fano_res=True)

#run the experiment
data = lab.run_single(res_spec, [(lab.VAR('res_freq'), np.arange(7.4050e9, 7.4175e9, 0.2e6))])

# Qubit Spectroscopy

In [35]:
#Variable for power supplied to the microwave resonator
stz.VariableProperty('qubit_power', lab, lab.HAL("mw_qubit"), 'Power')

#Variable for power supplied to the microwave resonator
stz.VariableProperty('qubit_freq', lab, lab.HAL("mw_qubit"), 'Frequency')

stz.ExperimentSpecification('Q1', lab, 'Qubit')
lab.SPEC('Q1').set_destination('Frequency GE', lab.VAR('qubit_freq'))
lab.SPEC('Q1').set_destination('Power', lab.VAR('qubit_power'))

In [ ]:
lab.CONFIG('contMeas').init_instruments()

lab.HAL('mw_qubit').Output = True
lab.HAL('mw_qubit').Mode = "Continuous" 
lab.HAL('mw_qubit').Power = -25

lab.HAL('DCoff_I').Voltage = 0
lab.HAL('DCoff_Q').Voltage = 0

stz.ExperimentConfiguration('qubit_spec', lab, 5e-6, ['DDG', 'mw_res', 'mw_dnc', 'wfm_res', 'mw_qubit'], 'TaborACQ', ['res_Q1'])

In [ ]:
exp_qubit_sweep = ExpPeakScouterIQ('qubit_spec_Q1', lab.CONFIG('qubit_spec'), param_centre=lab.SPEC('Q1')['Frequency GE'], is_trough=False)
data = lab.run_single(exp_qubit_sweep, [(lab.VAR('qubit_freq'), np.arange(5.0e9,5.2e9, 1.5e6))])

# Mixer Calibration

In [45]:
stz.WFMT_ModulationIQ('QubitFreqGE', lab, 100e6)

stz.ExperimentSpecification('MixerCalib', lab)
lab.SPEC('MixerCalib').add('IdcOff', 0, lab.HAL('DCoff_I'), 'Voltage')
lab.SPEC('MixerCalib').add('QdcOff', 0, lab.HAL('DCoff_Q'), 'Voltage')
lab.SPEC('MixerCalib').add('AmpFac', 0, lab.WFMT('QubitFreqGE'), 'IQAmplitudeFactor')
lab.SPEC('MixerCalib').add('PhsOff', 0, lab.WFMT('QubitFreqGE'), 'IQPhaseOffset')

stz.ExperimentSpecification('MixerCalibOff', lab)
lab.SPEC('MixerCalibOff').add('IdcOff', 0, lab.HAL('DCoff_I'), 'Voltage')
lab.SPEC('MixerCalibOff').add('QdcOff', 0, lab.HAL('DCoff_Q'), 'Voltage')
lab.SPEC('MixerCalibOff').add('AmpFac', 0, lab.WFMT('QubitFreqGE'), 'IQAmplitudeFactor')
lab.SPEC('MixerCalibOff').add('PhsOff', 0, lab.WFMT('QubitFreqGE'), 'IQPhaseOffset')

In [ ]:
lab.CONFIG('contMeas').init_instruments()

lab.HAL("mw_qubit").Power = 16
centre_freq = lab.SPEC('Q1')['Frequency GE'].Value + 100e6
lab.HAL('mw_qubit').Frequency = centre_freq

lab.VAR('drive_freq').Value = 100e6

stz.WaveformAWG("WfmConMixer", lab, [(['Tabor', 'AWG'], 'CH1'), (['Tabor', 'AWG'], 'CH2')], 2e9)
lab.HAL('WfmConMixer').set_valid_total_time(lab.HAL('DDG').RepetitionTime-0.1e-6)
lab.HAL("WfmConMixer").add_waveform_segment(stz.WFS_Constant("pad", lab.WFMT('QubitFreqGE').apply(), 512e-9, 0.2))
lab.HAL("WfmConMixer").add_waveform_segment(stz.WFS_Constant("pulse", lab.WFMT('QubitFreqGE').apply(), -1, 0.2))
lab.HAL("WfmConMixer").get_output_channel(0).marker(2).set_markers_to_segments(['pulse'])
lab.HAL("WfmConMixer").set_trigger_source_all(lab.HAL("DDG").get_trigger_output('AB'))
lab.HAL("TaborACQ").set_trigger_source(lab.HAL("WfmConMixer").get_output_channel(0).marker(2))

lab.HAL('swQBox').Position = 'Pmix'

stz.ExperimentConfiguration('mixer_calib', lab, 5e-6, ['DDG', 'mw_res', 'mw_dnc', 'WfmConMixer', 'mw_qubit', 'swQBox'], 'TaborACQ')
lab.CONFIG('mixer_calib').plot();

In [ ]:
stz.VariableProperty('dnc_freq', lab, lab.HAL("mw_dnc"), 'Frequency')

stz.VariableInternal('freq_mixer_calib', lab)
stz.VariableSpaced('qubit_freq_mixer_calib', lab, 'freq_mixer_calib', 'dnc_freq', -25e6)

new_exp = stz.Experiment('mixer_calib_sweep', lab.CONFIG('mixer_calib'))



mixer_calib_data = lab.run_single(new_exp, [(lab.VAR('qubit_freq_mixer_calib'), np.arange(-200e6, 200e6, 10e6)+centre_freq)])

In [ ]:
import matplotlib.pyplot as plt

arr = mixer_calib_data.get_numpy_array()

plt.plot(mixer_calib_data.param_vals[0], arr[:,0]**2+arr[:,1]**2)

## LO Suppression

In [16]:
stz.VariableProperty('DCOFFI', lab, lab.HAL('DCoff_I'), 'Voltage')
stz.VariableProperty('DCOFFQ', lab, lab.HAL('DCoff_Q'), 'Voltage')

#ramp rate
lab.HAL('DCoff_I').RampRate = 1
lab.HAL('DCoff_Q').RampRate = 1

In [ ]:
from sqdtoolz.Experiments.Experimental.ExpMixerCalibrationLO import ExpMixerCalibrationLO

new_exp = ExpMixerCalibrationLO('mixer_calib_LO', lab.CONFIG('mixer_calib'), 
                                lab.VAR('qubit_freq_mixer_calib'), 
                                centre_freq,
                                lab.VAR('DCOFFI'), (-0.4,0.4), lab.VAR('DCOFFQ'), (-1,-0.2))
LO_data = lab.run_single(new_exp)

In [ ]:
new_exp = ExpMixerCalibrationLO('mixer_calib_LO', lab.CONFIG('mixer_calib'), lab.VAR('qubit_freq_mixer_calib'), 
                                centre_freq,
                                lab.VAR('DCOFFI'), (-0.4,0.8), lab.VAR('DCOFFQ'), (-1,-0.2),
                                optimise=True, sample_points=4, iterations=10, win_shrink_factor=0.33, acq_ch = 'CH1_0')
LO_data_opt = lab.run_single(new_exp)

In [19]:
new_exp.commit_to_SPEC(lab.SPEC('MixerCalib'))

## SB Calibration

In [ ]:
from sqdtoolz.Experiments.Experimental.ExpMixerCalibrationSB import ExpMixerCalibrationSB

stz.VariableProperty('IQ_amp_fac', lab, lab.WFMT('QubitFreqGE'), 'IQAmplitudeFactor')
stz.VariableProperty('IQ_phs_diff', lab, lab.WFMT('QubitFreqGE'), 'IQPhaseOffset')

upper_SB_freq = centre_freq+100e6

new_exp = ExpMixerCalibrationSB(f'mixer_calib_SB', lab.CONFIG('mixer_calib'), lab.VAR('qubit_freq_mixer_calib'), upper_SB_freq,
                             lab.VAR('IQ_amp_fac'), (0.6,1.4), lab.VAR('IQ_phs_diff'), (2,4))
SB_data = lab.run_single(new_exp)

In [ ]:
new_exp = ExpMixerCalibrationSB('mixer_calib_SB', lab.CONFIG('mixer_calib'), lab.VAR('qubit_freq_mixer_calib'), upper_SB_freq,
                             lab.VAR('IQ_amp_fac'), (0.8,1.2), lab.VAR('IQ_phs_diff'), (2.5, 3.5), optimise=True, sample_points=4, iterations=10, win_shrink_factor=0.33, acq_ch = 'CH1_0')
SB_data_opt = lab.run_single(new_exp)

In [22]:
new_exp.commit_to_SPEC(lab.SPEC('MixerCalib'))

# PreRabi

In [25]:
stz.VariableInternal('qubit_freq_actual', lab)
stz.VariableSpaced('qubit_freq_LO', lab, 'qubit_freq_actual', 'qubit_freq', 100e6)
lab.SPEC('Q1').set_destination('Frequency GE', lab.VAR('qubit_freq_LO'))

In [ ]:
lab.CONFIG('contMeas').init_instruments()

lab.HAL("mw_qubit").Power = 16
lab.HAL('mw_qubit').Frequency = lab.SPEC('Q1')['Frequency GE'].Value

lab.HAL("DDG").RepetitionTime = 250e-6

stz.WaveformAWG("wfm_pre_rabi", lab, [(['Tabor', 'AWG'], 'CH1'), (['Tabor', 'AWG'], 'CH2')], 2e9)
lab.HAL('wfm_pre_rabi').set_valid_total_time(lab.HAL('DDG').RepetitionTime-0.1e-6)
lab.HAL("wfm_pre_rabi").add_waveform_segment(stz.WFS_Constant("pulse", lab.WFMT('QubitFreqGE').apply(), -1, 0.05))
lab.HAL("wfm_pre_rabi").add_waveform_segment(stz.WFS_Constant("read", lab.WFMT('QubitFreqGE').apply(), 4096e-9, 0.000))
lab.HAL("wfm_pre_rabi").get_output_channel(0).marker(2).set_markers_to_segments(['read'])
lab.HAL("wfm_pre_rabi").set_trigger_source_all(lab.HAL("DDG").get_trigger_output('AB'))
lab.HAL("TaborACQ").set_trigger_source(lab.HAL("wfm_pre_rabi").get_output_channel(0).marker(2))

lab.HAL('swQBox').Position = 'Pmeas'

stz.ExperimentConfiguration('pre_rabi', lab, lab.HAL("DDG").RepetitionTime, ['DDG', 'mw_res', 'mw_dnc', 'wfm_pre_rabi', 'mw_qubit', 'swQBox'], 'TaborACQ', ['res_Q1', 'MixerCalib', 'Q1'])
lab.CONFIG('pre_rabi').plot();

In [ ]:
stz.VariableProperty('drive_freq', lab, lab.WFMT('QubitFreqGE'), 'IQFrequency')

new_exp = stz.Experiment('prerabi_pulsed', lab.CONFIG('pre_rabi'))
prerabi_data = lab.run_single(new_exp, [(lab.VAR('drive_freq'), np.arange(70e6, 190e6, 1e6))])

# Rabi

In [ ]:
lab.CONFIG('pre_rabi').init_instruments()

lab.HAL("DDG").RepetitionTime = 500e-6
lab.HAL("DDG").get_trigger_output('AB').TrigPulseLength = 100e-9
lab.HAL("DDG").get_trigger_output('AB').TrigPolarity = 1
lab.HAL("DDG").get_trigger_output('AB').TrigPulseDelay = 0.0e-9

#Estimate valid time from a previous waveform (1us lower to ensure DDG rep time is greater than AWG waveform)
stz.WaveformAWG("wfmRabiA", lab, [(['Tabor', 'AWG'], 'CH1'), (['Tabor', 'AWG'], 'CH2')], 2e9)
lab.HAL("wfmRabiA").set_valid_total_time(lab.HAL("DDG").RepetitionTime-1e-6)
lab.HAL("wfmRabiA").add_waveform_segment(stz.WFS_Constant("init_pad", None, -1, 0.0))
lab.HAL("wfmRabiA").add_waveform_segment(stz.WFS_Constant("init", None, 40e-6, 0.0))
lab.HAL("wfmRabiA").add_waveform_segment(stz.WFS_Gaussian("drive", lab.WFMT('QubitFreqGE').apply(phase=0), 40e-9, 0.05))
lab.HAL("wfmRabiA").add_waveform_segment(stz.WFS_Constant("pad", None, 5e-9, 0.0))
lab.HAL("wfmRabiA").add_waveform_segment(stz.WFS_Constant("read", None, 4e-6, 0.0))
lab.HAL("wfmRabiA").get_output_channel(0).marker(0).set_markers_to_segments(['read'])
lab.HAL("wfmRabiA").get_output_channel(0).marker(2).set_markers_to_segments(['read'])
lab.HAL("wfmRabiA").AutoCompression = 'None'

lab.HAL("wfmRabiA").set_trigger_source_all(lab.HAL("DDG").get_trigger_output('AB'))
lab.HAL("TaborACQ").set_trigger_source(lab.HAL("wfmRabiA").get_output_channel(0).marker(2))
lab.HAL("mw_res").set_trigger_source(lab.HAL("wfmRabiA").get_output_channel(0).marker(0))

lab.HAL("mw_res").Mode = 'PulseModulated'

lab.HAL("mw_qubit").Power = 16
lab.HAL("mw_qubit").Mode = 'Continuous'

stz.ExperimentConfiguration('RabiAuto', lab, lab.HAL("DDG").RepetitionTime, ['DDG', 'mw_res', 'mw_dnc', 'wfmRabiA', 'mw_qubit', 'swQBox'], 'TaborACQ', ['res_Q1', 'MixerCalib', 'Q1'])


#########################????????????????????
lab.CONFIG('RabiAuto').plot();
#########################

waveform_mapping = stz.WaveformMapper()
waveform_mapping.add_waveform('qubit', 'wfmRabiA')
waveform_mapping.add_digital('readout', [lab.HAL("wfmRabiA").get_output_channel(0).marker(2), lab.HAL("wfmRabiA").get_output_channel(0).marker(0)])
# waveform_mapping.add_digital('sequence', awg_wfm.get_output_channel(1).marker(0))
lab.CONFIG('RabiAuto').map_waveforms(waveform_mapping)

In [ ]:
lab.WFMT('QubitFreqGE').IQFrequency = 80e6
lab.SPEC('res_Q1')['Power'].Value = -20

In [ ]:
from sqdtoolz.Experiments.Experimental.ExpRabi import ExpRabi

stz.VariableInternal('RabiFreqAmpl', lab)

# lab.WFMT('QubitFreqGE').IQFrequency = 100e6

new_exp = ExpRabi(f"RabiAuto", lab.CONFIG('RabiAuto'), lab.WFMT('QubitFreqGE'), np.linspace(0, 0.5, 30), lab.SPEC('Q1'),
                  param_rabi_frequency=lab.VAR('RabiFreqAmpl'), load_time = 80e-6, drive_time= 40e-9, readout_time = 4.096e-6, normalise=False, normalise_reps=2)
rabi_data = lab.run_single(new_exp)

In [ ]:
from sqdtoolz.Experiments.Experimental.ExpRabi import *

stz.VariableInternal('RabiFreqAmpl', lab)
new_exp = ExpRabi(f"RabiAuto", lab.CONFIG('RabiAuto'), lab.WFMT('QubitFreqGE'), np.linspace(0, 0.5, 30), lab.SPEC('Q1'),
                  param_rabi_frequency=lab.VAR('RabiFreqAmpl'), load_time = 80e-6, drive_time= 40e-9, readout_time = 4.096e-6, normalise=True, normalise_reps=2)
rabi_data = lab.run_single(new_exp)

# Ramsey

In [ ]:
from sqdtoolz.Experiments.Experimental.ExpRamseyGE import ExpRamseyGE

lab.WFMT('QubitFreqGE').IQFrequency += 2e6

stz.VariableInternal('RamseyFreq', lab)
stz.VariableInternal('RamseyTau', lab)

new_exp = ExpRamseyGE(f"RamseyAuto_short", lab.CONFIG('RabiAuto'), lab.WFMT('QubitFreqGE'), 
                    np.linspace(0,1e-6,30), 
                    lab.SPEC('Q1'), param_ramsey_frequency=lab.VAR('RamseyFreq'), 
                    param_ramsey_decay_time=lab.VAR('RamseyTau'),
                    load_time=80e-6, readout_time=4.096e-6, normalise= False, normalise_reps=2)
ramsey_data = lab.run_single(new_exp)

lab.WFMT('QubitFreqGE').IQFrequency = lab.WFMT('QubitFreqGE').IQFrequency - lab.VAR('RamseyFreq').Value

# print('New IQ Frequency:')
# print(lab.WFMT('QubitFreqGE').IQFrequency)

In [ ]:
from sqdtoolz.Experiments.Experimental.ExpRamseyGE import ExpRamseyGE

lab.WFMT('QubitFreqGE').IQFrequency += 0.1e6

stz.VariableInternal('RamseyFreq', lab)
stz.VariableInternal('RamseyTau', lab)

new_exp = ExpRamseyGE(f"RamseyAuto_short", lab.CONFIG('RabiAuto'), lab.WFMT('QubitFreqGE'), 
                    np.linspace(0,40e-6,80), 
                    lab.SPEC('Q1'), param_ramsey_frequency=lab.VAR('RamseyFreq'), 
                    param_ramsey_decay_time=lab.VAR('RamseyTau'),
                    load_time=80e-6, readout_time=4.096e-6, normalise= True, normalise_reps=2)
ramsey_data = lab.run_single(new_exp)

lab.WFMT('QubitFreqGE').IQFrequency = lab.WFMT('QubitFreqGE').IQFrequency - lab.VAR('RamseyFreq').Value

print(lab.VAR('RamseyTau').Value)

# T1

In [ ]:
from sqdtoolz.Experiments.Experimental.ExpT1GE import ExpT1GE

stz.VariableInternal('QubitT1', lab)

new_exp = ExpT1GE(f"T1Auto", lab.CONFIG('RabiAuto'), lab.WFMT('QubitFreqGE'), np.linspace(0, 200e-6, 30), lab.SPEC('Q1'),
                param_T1_decay_time = lab.VAR('QubitT1'), load_time=80e-6, readout_time=4.096e-6, normalise= True, normalise_reps=4)

leData = lab.run_single(new_exp)

print('T1:')
print(lab.VAR('QubitT1').Value)